In [1]:
import luigi
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '/home/fbuonerba/codes/')
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
import calendar
import json
import urllib.request
import multiprocessing as mp
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
#####January to April#####

from mp_functions import upload_log_return
with open('/home/fbuonerba/codes/meta_data/top_coins.txt') as ff:
    coins=json.load(ff)
coins=list(coins)
####LIZA is outlier for 2 of the risk factors...####
coins.remove('LIZA')
quotes=['USD','BTC']
#sysm=[str(coin)+'_'+str(quote) for coin in coins for quote in quotes]
t=1515283200
matrix=[]
#matrix of daily log_returns (time, base_quote)
while t<=1522540801:
    ret_t=[]
    for base in coins:
        for quote in quotes:
            returns=upload_log_return(t, base, quote, 86400)
            if np.isnan(returns)==True:
                returns=0
            ret_t.append(returns)
            #print(base,quote)
    matrix.append(ret_t)
    t+=86400
R=np.array(matrix)

factors=[]
factor_names1=['variance_', 'strength_', 'high_low_']
factor_names2=['turnover_', 'log_mkcap_']
factor_names3=['coin_ratio_']
path='/home/fbuonerba/codes/factor_loadings/'
finpath1='_'+str(1515283200)+'_'+str(1522540800)+'_'+str(604800)+'.txt'
finpath2='_'+str(1515283200)+'_'+str(1522540800)+'.txt'
finpath3='.txt'
for base in coins:
    for quote in quotes:
        sym=str(base)+'_'+str(quote)
        sym_row=[]
        for name in factor_names1:
            with open(path+name+sym+finpath1) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names2:
            with open(path+name+sym+finpath2) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names3:
            #symll=sym.split('_')
            with open(path+name+base+finpath3) as file:
                x=json.load(file)
            sym_row.append(x)
        sym_row.append(1)
        factors.append(sym_row)
factors=np.array(factors)  


In [3]:
reg = linear_model.LinearRegression()
reg.fit(factors, R.T)
beta_=reg.coef_
inter_=reg.intercept_
R_hat=np.dot(factors,beta_.T)+inter_
print(mean_squared_error(R.T, R_hat), reg.score(factors, R.T), r2_score(R.T,R_hat))


0.010712758522135874 0.24808812665660848 0.2571971858182343


In [5]:
#####February to May#####

from mp_functions import upload_log_return
with open('/home/fbuonerba/codes/meta_data/top_coins.txt') as ff:
    coins=json.load(ff)
coins=list(coins)
####LIZA is outlier for 2 of the risk factors...####
coins.remove('LIZA')
quotes=['USD','BTC']
#sysm=[str(coin)+'_'+str(quote) for coin in coins for quote in quotes]
t=1517702400
matrix=[]
#matrix of daily log_returns (time, base_quote)
while t<=1525564801:
    ret_t=[]
    for base in coins:
        for quote in quotes:
            returns=upload_log_return(t, base, quote, 86400)
            if np.isnan(returns)==True:
                returns=0
            ret_t.append(returns)
            #print(base,quote)
    matrix.append(ret_t)
    t+=86400
R=np.array(matrix)

factors=[]
factor_names1=['variance_', 'strength_', 'high_low_']
factor_names2=['turnover_', 'log_mkcap_']
factor_names3=['coin_ratio_']
path='/home/fbuonerba/codes/factor_loadings/'
finpath1='_'+str(1517702400)+'_'+str(1525564800)+'_'+str(604800)+'.txt'
finpath2='_'+str(1517702400)+'_'+str(1525564800)+'.txt'
finpath3='.txt'
for base in coins:
    for quote in quotes:
        sym=str(base)+'_'+str(quote)
        sym_row=[]
        for name in factor_names1:
            with open(path+name+sym+finpath1) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names2:
            with open(path+name+sym+finpath2) as file:
                x=json.load(file)
            sym_row.append(x)
        for name in factor_names3:
            #symll=sym.split('_')
            with open(path+name+base+finpath3) as file:
                x=json.load(file)
            sym_row.append(x)
        sym_row.append(1)
        factors.append(sym_row)
factors=np.array(factors)   
#X_with_ones = np.hstack((np.ones((factors.shape[0], 1)), factors))

In [7]:
reg = linear_model.LinearRegression()
reg.fit(factors, R.T)
beta_=reg.coef_
inter_=reg.intercept_
#print(beta_)
R_hat1=reg.predict(factors)
R_hat=np.dot(factors,beta_.T)+inter_
print(reg.score(factors,R.T),r2_score(R.T,R_hat), r2_score(R.T,R_hat1,multioutput='variance_weighted'))
#reg.score computes the mean along axis=0
#r2_score computes the mean over all array


0.16324635914651683 0.21790209793205853 0.16324635914651683


In [43]:
#factors must have extra column of ones for this to work.
beta=np.linalg.lstsq(factors, R.T,rcond=None)
gamma=beta[0]
R_hat_np=np.dot(factors,gamma)
residuals=np.sum( (R.T-R_hat_np)**2)
#average returns over days, for each coin
mean=np.mean(R, axis=0)
print(mean.shape)
sum_squares=np.sum((R-mean.T)**2)
r2=1-residuals/sum_squares
#average returns over coins, for each day.
mean_bs=np.mean(R, axis=1)
print(mean_bs.shape)
sum_squares_bs=np.sum((R.T-mean_bs)**2)
r2_bs=1-residuals/sum_squares_bs
print(r2, r2_bs)
#just average over days and coins
1-residuals/np.linalg.norm(R.T-np.mean(R))**2

(88,)
(92,)
0.27034767080001776 0.16324635914651675


0.27162467611784746

In [ ]:
######################################################

In [ ]:
#regression against each one singule time, BTC/USD
factor_test=factors
r=[]
for i in range(85):
    R_test=R[i]
    beta_test=np.linalg.lstsq(factor_test,R_test.T, rcond=None)
    S=np.dot(factors,beta_test[0])
    res_test=np.linalg.norm(R_test-S)**2
    r.append(1-res_test/np.linalg.norm(R_test)**2)
r=np.array(r)
print(np.max(r), np.min(r))

In [ ]:
temp=R[83]
factt=factors.T[6]
fac=np.vstack((factt, np.ones_like(factt)))

temp_beta=np.linalg.lstsq(fac.T,temp,rcond=None)
print(temp_beta[0])
import matplotlib.pyplot as plt
plt.plot(factt, temp,'o')
plt.plot(factt, temp_beta[0][0]*factt + temp_beta[0][1], 'r', label='Fitted line')
plt.show()

In [ ]:
for t in range(88):
    print(coins[int(t/2)],quotes[t%2])
    plt.plot(range(85), R.T[t])
    plt.show()